# imorting files

In [1]:
import pandas as pd

In [2]:
links = pd.read_csv('csv_files/links.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies = pd.read_csv('csv_files/movies.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings = pd.read_csv('csv_files/ratings.csv')

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
tags =  pd.read_csv('csv_files/tags.csv')

In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [10]:
df = pd.merge(ratings, movies, on='movieId')

In [11]:
new_df = ratings.drop(columns='timestamp')

# Recommendation System

In [12]:
from surprise import Reader, Dataset
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split

In [13]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)

In [14]:

benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.868845,882.926506,32.773629
BaselineOnly,0.876045,0.222493,0.456649
SVD,0.879474,9.959715,0.614917
KNNBaseline,0.881853,0.539366,7.746760
KNNWithMeans,0.905121,0.386333,6.592975
KNNWithZScore,0.905275,0.452219,6.905525
SlopeOne,0.912581,6.849113,21.746805
NMF,0.934993,11.795354,0.525171
CoClustering,0.955518,5.107392,0.581912


In [15]:
#need to grid search
#cross_validate(SVDpp(), data, measures=['RMSE'], cv=3, verbose=False)

In [16]:
trainset, testset = train_test_split(data, test_size=0.25)
predictions = SVDpp().fit(trainset).test(testset)

RMSE: 0.8653


0.8653077081786326

In [21]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['est'] =  round(df['est'] * 2) / 2
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [22]:
df.head()

,uid,iid,rui,est,details,Iu,Ui,err
0,480,466,0.5,2.5,{'was_impossible': False},619,40,2.0
1,573,8636,4.0,5.0,{'was_impossible': False},224,57,1.0
2,119,4718,4.0,3.5,{'was_impossible': False},163,34,0.5
3,474,3037,4.0,4.0,{'was_impossible': False},1606,19,0.0
4,414,1562,2.0,1.5,{'was_impossible': False},2019,32,0.5


In [23]:
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [24]:
 df.loc[df['uid'] == 1].sort_values(by='err').head()

,uid,iid,rui,est,details,Iu,Ui,err
73,1,2174,4.0,4.0,{'was_impossible': False},177,66,0.0
22138,1,367,4.0,4.0,{'was_impossible': False},177,110,0.0
22107,1,3147,5.0,5.0,{'was_impossible': False},177,89,0.0
20445,1,2571,5.0,5.0,{'was_impossible': False},177,212,0.0
19158,1,527,5.0,5.0,{'was_impossible': False},177,161,0.0


In [25]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list     

In [26]:
user_rating = movie_rater(movies, 4, 'Action')

      movieId                         title                  genres
7407    80162  Horde, The (La Horde) (2009)  Action|Horror|Thriller
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                       title           genres
3481     4750  3 Ninjas Knuckle Up (1995)  Action|Children
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
      movieId                                           title  \
3997     5638  Godzilla vs. Mothra (Mosura tai Gojira) (1964)   

                               genres  
3997  Action|Adventure|Fantasy|Sci-Fi  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
2
      movieId                        title                    genres
4872     7302  Thief of Bagdad, The (1924)  Action|Adventure|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1


In [27]:
new_ratings_df = new_df.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [28]:
# train a model using the new combined DataFrame
svd_ = SVDpp(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [29]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [30]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [31]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,movies,5)

Recommendation #  1 :  937    Seventh Seal, The (Sjunde inseglet, Det) (1957)
Name: title, dtype: object 

Recommendation #  2 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 

Recommendation #  3 :  266    Three Colors: Red (Trois couleurs: Rouge) (1994)
Name: title, dtype: object 

Recommendation #  4 :  918    Ran (1985)
Name: title, dtype: object 

Recommendation #  5 :  796    Secrets & Lies (1996)
Name: title, dtype: object 

